<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/InsightFace_mesen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls -d /usr/local/cuda-*
!nvcc --version

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu2004-11-0-local/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda-11-0

In [ ]:
!ls -d /usr/local/cuda-*
!nvcc --version

In [5]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.0/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.0/lib64:{ld}"

In [ ]:
!ls -d /usr/local/cuda-*
!nvcc --version

In [ ]:
!pip install insightface

In [ ]:
#!pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
!pip install onnxruntime-gpu

In [ ]:
from insightface.app import FaceAnalysis

In [ ]:
!pip install moviepy

In [ ]:
!pip install numpy==1.23.5              #最新VerではInseight実行時にエラーが出るためこのバージョンに下げる。このVerでは初回Import時に失敗する。 ランタイムの再起動も発生。

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from insightface.app import FaceAnalysis                      #もう一度インポート
from moviepy.editor import VideoFileClip, AudioFileClip

In [16]:
class INSIGHTFACE(object):
    def __init__(self, ctx_id=0, det_size_x=640, det_size_y=640):
        #self.app = FaceAnalysis(root='./.insightface')
        self.app = FaceAnalysis(root='./.insightface', providers=['CUDAExecutionProvider'])
        self.app.prepare(ctx_id=ctx_id, det_size=(det_size_x, det_size_y))
        self.faces = []

    def get_InsightFace(self, img):
        self.faces = self.app.get(np.asarray(img))
        #for face in faces:
        #    kps   = face.kps.astype(np.int)
        #    print(kps)
        rimg  = self.app.draw_on(img, self.faces)
        #cv2.imwrite("./output.jpg", rimg)
        return img, rimg, self.faces

    def draw_mesen(self, img):
        KPS = []
        for face in self.faces:
            kps = face.kps.astype(np.int)
            KPS.append(kps)
        EYE = _EYE = [[]] * 2
        out_img = img
        for _eye in KPS:
            EYE[0] = _eye[0]
            EYE[1] = _eye[1]
            lineSize  = abs(np.linalg.norm(np.array(EYE[0])-np.array(EYE[1])))
            lineWidth = int(lineSize / 4)
            if lineWidth <= 10:
                lineWidth = 10
            lineAngle = 1
            if EYE[1][0]-EYE[0][0] != 0:
                lineAngle   = (EYE[1][1]-EYE[0][1])/(EYE[1][0]-EYE[0][0])
            _EYE[0][0] = EYE[0][0] - (lineSize / 3)
            _EYE[0][1] = lineAngle * (_EYE[0][0] - EYE[0][0]) + EYE[0][1]
            _EYE[1][0] = EYE[1][0] + (lineSize / 3)
            _EYE[1][1] = lineAngle * (_EYE[1][0] - EYE[0][0]) + EYE[0][1]
            out_img = cv2.line(out_img,
                              (_EYE[0][0], _EYE[0][1]),
                              (_EYE[1][0], _EYE[1][1]),
                              (0,0,0),
                              thickness=lineWidth)
        return out_img

In [ ]:
IF = INSIGHTFACE()

In [ ]:
import onnxruntime as ort

print(ort.get_device())
print(ort.get_available_providers())

provider = ['CUDAExecutionProvider','CPUExecutionProvider']
provider = ['CUDAExecutionProvider']
ort_sess = ort.InferenceSession('./.insightface/models/buffalo_l/w600k_r50.onnx', providers=provider)
print(ort_sess.get_providers())

In [10]:
!cp ./drive/MyDrive/zoom_480p.mov ./

In [ ]:
video_name = "zoom_480p.mov"
video_path = "./" + video_name
mask_path  = '.'.join(video_path.split('.')[:-1]) + '_mask.mp4'
audio_path = '.'.join(video_path.split('.')[:-1]) + '.wav'
output_path = '.'.join(video_path.split('.')[:-1]) + '_audio_mask.mp4'
video = VideoFileClip(video_path)
video.audio.write_audiofile(audio_path, ffmpeg_params=["-ac", "1"])  #モノラルで出力

In [12]:
cap = cv2.VideoCapture(video_path)
if cap.isOpened() is False:
    cap = None

In [13]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = float(cap.get(cv2.CAP_PROP_FPS))
frameNum = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [14]:
codec = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(mask_path, codec, fps, (width, height), True)

In [ ]:
bar = tqdm(total=frameNum, dynamic_ncols=True)
cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
ret, img = cap.read()
while ret :
    input_img, draw_img, faces = IF.get_InsightFace(img)
    out_img = IF.draw_mesen(input_img)
    writer.write(out_img)
    ret, img = cap.read()
    bar.update(1)
writer.release()

In [ ]:
video_clip = VideoFileClip(mask_path)
audio_clip = AudioFileClip(audio_path)
video_clip = video_clip.set_audio(audio_clip)
video_clip.write_videofile(output_path, audio_codec = 'aac')

In [ ]:
! ls -alh

In [ ]:
!cp ./zoom_480p_mask.mp4 ./drive/MyDrive
!cp ./zoom_480p.mp3 ./drive/MyDrive
!cp ./zoom_480p_audio_mask.mp4 ./drive/MyDrive